In [ ]:
%matplotlib inline

## 신경망모델 구성

신경망은 데이터 연산을 수행하는 계층 layer / 모듈 module 로 구성되어 있다.

torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

##학습장치 설정

GPU or MPS 없다면 CPU

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

##클래스




In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self): # 신경망 계층 초기화
        super().__init__()
        self.flatten = nn.Flatten() # 1차원 평탄화
        self.linear_relu_stack = nn.Sequential(   # 레이어 쌓기
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Neuralnetwork 인스턴스를 생성하고 device로 이동 후 structure 출력

In [ ]:
model = NeuralNetwork().to(device)
print(model)

입력 데이터 전달, forward 실행

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환

dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개  
dim=1에는 각 출력의 개별 값

원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다



In [ ]:
X = torch.rand(1, 28, 28, device = device) # 입력 데이터 생성
logits = model(X) # 모델에 입력 데이터 넣고 각 클래스에 대한 logit값 저장
pred_probab = nn.Softmax(dim=1)(logits) # 로짓을 소프트맥스 함수로 확률값 변환
y_pred = pred_probab.argmax(1) # 가장 높은 확률이 예측된 클래스 반환
print(f"Predicted class: {y_pred}")

##모델 계층

FashionMNIST 모델의

28 X 28 크기의 이미지 3개로 구성된 미니배치 로 신경망 통과과정 살피기


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

## nn.Flatten

평탄화 계층 -> 계층 초기화 이후 28x28 2d 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환

In [ ]:
flatten= nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

##nn.Linear

선형계층 -> 저장된 가중치, 편향 사용해서 입력에 선형변환을 적용

In [ ]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20) # 선형변환!
hidden1 = layer1(flat_image)
print(hidden1.size())

##nn.ReLU

비선형 활성화는 모델의 입출력 사이에 복잡한 관계(mapping) 만듬

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
# 양수는 자신, 음수는 0
print(f"After ReLU: {hidden1}")

## nn.Sequential

순서 정해주기

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## 모델 매개변수

신경망 내부 계층들을 매개변수화 함 -> 학습 중 최적화 되는 가중치 & 편향과 연관지어짐
nn.module 쓰면 모델 내 필드들이 자동으로 추적된다. parameters(), named_parameters() 메소드로 모든 배개변수에 접근 가능해짐

In [ ]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")